# Discount Cash Flow Calculation
This Python script performs a financial valuation of a company (in this case, with the ticker symbol 'AAPL') using discounted cash flow (DCF) analysis. Here's a breakdown of the steps involved:

1. **fetch_financial_data(ticker)**:
   - This function takes a ticker symbol as input and attempts to fetch financial data using the Yahoo Finance API (`yf.Ticker(ticker)`).
   - It retrieves the company's cash flow statement and extracts the free cash flow.
   - It also obtains the number of outstanding shares of the company.
   - Any errors encountered during this process are handled and appropriate error messages are printed.

2. **calculate_cashflow_growth_rate(free_cash_flows)**:
   - This function calculates the average annual growth rate of free cash flows using percentage change.

3. **project_future_free_cash_flows(last_cash_flow, growth_rate, years)**:
   - Given the last observed free cash flow, growth rate, and a specified number of years, this function projects future free cash flows using a compounding growth formula.

4. **discount_cash_flows(future_cash_flows, required_rate)**:
   - This function discounts the projected future cash flows back to their present value using a discount factor based on the required rate of return (WACC).

5. **calculate_terminal_value(last_cash_flow, growth_rate, required_rate, years)**:
   - It calculates the terminal value of the company, representing its value beyond the explicit forecast period, using the Gordon Growth Model.

6. **calculate_fair_value(discounted_cash_flows, terminal_value, outstanding_shares)**:
   - This function computes the fair value per share by summing the present values of discounted future cash flows and the terminal value, then dividing by the total number of outstanding shares.

7. **main()**:
   - Sets up parameters such as the required rate of return, perpetual growth rate, and the number of years to project.
   - Calls the above functions in sequence to perform the valuation.
   - Prints out the fair value per share of the company based on the DCF analysis.

This script provides a basic framework for valuing a company's stock by estimating its future cash flows and discounting them back to their present value, accounting for both the explicit forecast period and the terminal value.
}

In [ ]:
def fetch_financial_data(ticker):
    try:
        ticker_data = yf.Ticker(ticker)
        cash_flows_statement = ticker_data.cashflow
        balance_sheet = ticker_data.balance_sheet
        income_statement = ticker_data.financials
        
        free_cash_flows = cash_flows_statement.loc['Free Cash Flow']
        outstanding_shares = ticker_data.info['sharesOutstanding']

        # Additional data needed for WACC calculations
#         total_debt = balance_sheet.loc['Total Debt'][0]  # Latest total debt
#         total_equity = ticker_data.info['marketCap'] - total_debt  # Assuming marketCap approximates equity value
#         interest_expense = income_statement.loc['Interest Expense'][0]  # Latest interest expense
#         tax_expense = income_statement.loc['Income Tax Expense'][0]  # Latest tax expense
#         effective_tax_rate = tax_expense / (income_statement.loc['Pre-Tax Income'][0])  # Effective tax rate calculation

        
        return free_cash_flows, outstanding_shares
    except KeyError as e:
        print(f"A KeyError occurred: {e}. Check the labels used to extract data.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}.")

def calculate_cashflow_growth_rate(free_cash_flows):
    return free_cash_flows.pct_change().mean()

def project_future_free_cash_flows(last_cash_flow, growth_rate, years):
    return [last_cash_flow * (1 + growth_rate) ** i for i in range(1, years + 1)]

def discount_cash_flows(future_cash_flows, required_rate):
    discount_factors = [(1 + required_rate) ** i for i in range(1, len(future_cash_flows) + 1)]
    return [fcf / df for fcf, df in zip(future_cash_flows, discount_factors)]

def calculate_terminal_value(last_cash_flow, growth_rate, required_rate, years):
    return last_cash_flow * (1 + growth_rate) / (required_rate - growth_rate) / (1 + required_rate) ** years

def calculate_fair_value(discounted_cash_flows, terminal_value, outstanding_shares):
    total_present_value = sum(discounted_cash_flows) + terminal_value
    return total_present_value / outstanding_shares

def get_cost_of_equity(risk_free_rate, beta, market_return):
    return risk_free_rate + beta * (market_return - risk_free_rate)

def get_cost_of_debt(interest_rate, tax_rate):
    return interest_rate * (1 - tax_rate)

def get_proportions(market_value_equity, market_value_debt):
    total_value = market_value_equity + market_value_debt
    return market_value_equity / total_value, market_value_debt / total_value

def calculate_wacc(cost_of_equity, cost_of_debt, equity_proportion, debt_proportion, tax_rate):
    wacc = (cost_of_equity * equity_proportion) + ((1 - tax_rate) * cost_of_debt * debt_proportion)
    return wacc


def main():
    # Placeholder values for the purpose of this example
    risk_free_rate = 0.02
    beta = 1.2
    market_return = 0.08
    interest_rate = 0.04
    tax_rate = 0.21
    market_value_equity = 2000000000000  # Example value
    market_value_debt = 100000000000  # Example value
    
    ticker = 'AAPL'
    required_rate = 0.08 #calculate_wacc()
    perpetual_growth_rate = 0.02  # Long-term growth rate
    years = 2  # Number of years to project
    
    free_cash_flows, outstanding_shares = fetch_financial_data(ticker)
    cashflow_growth_rate = calculate_cashflow_growth_rate(free_cash_flows)
    future_cash_flows = project_future_free_cash_flows(free_cash_flows[-1], cashflow_growth_rate, years)
    discounted_future_cash_flows = discount_cash_flows(future_cash_flows, required_rate)
    terminal_value = calculate_terminal_value(future_cash_flows[-1], perpetual_growth_rate, required_rate, years)
    fair_value_per_share = calculate_fair_value(discounted_future_cash_flows, terminal_value, outstanding_shares)
    
    print(f"The fair value per share for {ticker} is: ${fair_value_per_share:.2f}")

if __name__ == "__main__":
    main()
